# ASL Model Training

fvjr156

1. Import required packages.

In [ ]:
!pip install tensorflow roboflow ultralytics opencv-python pillow numpy scikit-learn

2. Configure CUDA, loglevel, and memory growth.

In [ ]:
import matplotlib.pyplot as pyplot
import tensorflow as tf
import torch
import os

print(f'TensorFlow ver: {tf.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'Device count: {torch.cuda.device_count()}')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)

      # memory depends on GPU, you should adjust it as necessary
      tf.config.experimental.set_virtual_device_configuration(
          gpus[0],
          [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6144)]  # 6GB limit
      )
  except RuntimeError as e:
    print(e)
else:
  print('No CUDA devices. Will use CPU instead')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # to reduce verbose logging
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

3. Allow access to your drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

4. Collect all labels and files from specified directory

In [ ]:
import os
import numpy as np

data_dir = '/content/drive/MyDrive/asltraining/data/landmark_sequences/'
class_labels = sorted(os.listdir(data_dir))  # folder names => gesture class labels

sequences = []
sample_labels = []  # per-sample label list

print(f'Found {len(class_labels)} labels: {class_labels}')

for label in class_labels:
    label_dir = os.path.join(data_dir, label)
    for filename in os.listdir(label_dir):
        if filename.endswith('.npy'):
            print(f'Found {filename}')
            seq = np.load(os.path.join(label_dir, filename))
            sequences.append(seq)
            sample_labels.append(label)

sequences = np.array(sequences)  # shape: (num_samples, 30, 14  6)
sample_labels = np.array(sample_labels)
print(f'Found {len(sample_labels)} labels: {sample_labels}')


5. Encode labels. Map string labels to integers then to one-hot vector. Train/Test split.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_int_encoded = label_encoder.fit_transform(sample_labels)
label_categorical = to_categorical(label_int_encoded)

x_train, x_test, y_train, y_test = train_test_split(
    sequences, label_categorical, test_size=0.2, random_state=42
    )
# here, we split test and train datasets (test takes 20%, train takes 80%)

6. Configure LSTM model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Input(shape=(30, 146)))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(label_categorical.shape[1], activation='softmax'))

7. Compile the LSTM model

In [ ]:
# stops early with validation loss plataus or increase
early_stopping = EarlyStopping(
    monitor = 'val_accuracy',
    patience = 5,
    restore_best_weights = True
    )

checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

7a. Fine tuning model

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# model = tf.keras.models.load_model('/content/drive/MyDrive/asltraining/models/asl_model_lstm.keras')

# early_stopping = EarlyStopping(
#     monitor = 'val_accuracy',
#     patience = 5,
#     restore_best_weights = True
#     )

# checkpoint = ModelCheckpoint(
#     'best_model.keras',
#     monitor='val_accuracy',
#     save_best_only=True,
#     verbose=1
# )

8. Then we train. Analyze training curves when finished. Modify epochs to your preferred outcome.

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, checkpoint]
)

pyplot.plot(
    history.history['accuracy'],
    label = 'Train Accuracy'
    )
pyplot.plot(
    history.history['val_accuracy'],
    label = 'Validation Accuracy'
    )
pyplot.plot(
    history.history['loss'],
    label = 'Train Loss'
    )
pyplot.plot(
    history.history['val_loss'],
    label = 'Validation Loss'
    )
pyplot.legend()
pyplot.title("Training Curves")
pyplot.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

9. Evaluate model then save to TensorFlow and TFLite

In [ ]:
# evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')

# # saving the model in TensorFlow SavedModel format
# model.export('/content/drive/MyDrive/asltraining/models/asl_model_lstm')
# print("TF saved success.")

# saving the model in Keras v3 native format .keras
model.save('/content/drive/MyDrive/asltraining/models/asl_model_lstm.keras')
print("Keras saved success.")

In [ ]:
from tensorflow import lite
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/asltraining/models/asl_model_lstm.keras')
best_model = load_model('best_model.keras')

converter = lite.TFLiteConverter.from_keras_model(model)
converter2 = lite.TFLiteConverter.from_keras_model(best_model)

converter.experimental_enable_resource_variables = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,       # TFLite native ops
    tf.lite.OpsSet.SELECT_TF_OPS          # Add fallback to full TF ops (e.g., TensorList)
]
converter._experimental_lower_tensor_list_ops = False  # Important for LSTM/GRU support

converter2.experimental_enable_resource_variables = True
converter2.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter2._experimental_lower_tensor_list_ops = False

# add optimization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter2.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
tflite_model2 = converter2.convert()

with open('/content/drive/MyDrive/asltraining/models/asl_model_lstm_quant.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model saved successfully.")

with open('/content/drive/MyDrive/asltraining/models/asl_model_lstm_quant_best_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model (from best_model.keras) saved successfully.")

Optional: Confusion matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
cm = confusion_matrix(
    np.argmax(y_test, axis=1),
    np.argmax(y_pred, axis=1)
)

sns.heatmap(cm, annot=True, xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('ASL Gesture Confusion Matrix')
plt.figure(figsize=(12, 12))
plt.show()

In [ ]:
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/asltraining/models/asl_model_lstm_quant.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)